#Analyse de la reconstruction par SubtleMR


##Importation des librairies nécessaires

In [ ]:
!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
import glob
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0.

%matplotlib inline
%pip install xlrd

### INSTALL GHOSTSCRIPT (Required to display NLTK trees) ###
!apt install ghostscript python3-tk

In [1]:
%pip install path
%pip install rt_utils
%pip install monai
%pip install nest_asyncio
%pip install pydicom
%pip install xlwt
%pip install xlrd

import os
import glob
import xlrd
import numpy as np
from PIL import Image
from typing import TypeVar, Union
import cv2 as cv
import numpy as np
import tqdm
from tqdm.notebook import tqdm
from pydicom import dcmread
import pydicom as dicom
from path import Path
import rt_utils
import cv2 as cv
from monai.transforms import LoadImage
import pandas as pd
from skimage import data, img_as_float
from skimage.metrics import structural_similarity
from skimage.metrics import mean_squared_error
from skimage import color
from skimage.morphology import dilation
from skimage.color import rgb2lab, lab2rgb
from scipy.stats import entropy
from scipy import stats
from scipy.stats import mstats
from sklearn import metrics
import sys
from os import listdir
from sklearn.metrics import roc_auc_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive

drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


##Les variables contenant les chemins


In [3]:
patient_Folder= 'gdrive/MyDrive/BaseDonneeAnalyseSubtle'
pathFileSave = 'gdrive/MyDrive/FichierStatistique/'

In [4]:
%ls gdrive/MyDrive/BaseDonneeAnalyseSubtle/

202107484_REC/  202107484_REF/  202108484_REC/  202108484_REF/


##Les fonctions utilitaires


###Fonction en lien avec les images

In [5]:
def readImageMasks(slices_folder, mask_file):
  rt_struct = rt_utils.RTStructBuilder.create_from(slices_folder, mask_file)
  mask=[]
  mask_name = []
  print(rt_struct.get_roi_names())
  for i in rt_struct.get_roi_names():
      if ("GTV" in i) or ("M_1" in i) or ("M1" in i):
          mask.append(rt_struct.get_roi_mask_by_name(i).astype(np.float32))
          roiName = i
          roiName = roiName.replace(".", "_")
          mask_name.append(roiName)
  img = LoadImage("PydicomReader", image_only=True, force=True)(slices_folder).numpy()
  img = np.rot90(img)
  img= np.flip(img, 0)
  del rt_struct
  return img, mask, mask_name

def findSliceForOneMask(mask):
    intervalMask =[]
    for k in tqdm(range (0,mask.shape[2])):
        if 1 in mask[:,:,k]:
            intervalMask.append(k)
    return intervalMask

def affichage(img):
    fig = plt.figure(figsize=(10,10))
    plt.imshow(img, cmap='gray')
    plt.show()

def saveImage(img, savepath):
    img.save(savepath + "mask.png")

def dilatation(mask,iterate):
    for i in range (0,iterate):
        mask = dilation(mask)
    return mask

def miseDansMRFichier(name, patient_folder, patient):
    try:
        if os.path.exists(os.path.join(patient_folder, patient+ name,  "MR")!=True):
            os.mkdir(os.path.join(patient_folder, patient+ name,  "MR"))
    except:
        print('Le dossier MR existe déjà dans',name)
    for file in glob.glob(os.path.join(patient_folder, patient+ name,  "MR*")):
        if os.path.isfile(file):
            os.rename(file, os.path.join(patient_folder, patient+ name,  "MR", os.path.split(file)[1]))
    slices_folderRef = patient_folder+'/' + patient + name +'/MR'
    mask_fileRef = ""
    for m in glob.glob(os.path.join(patient_folder, patient+ name,  "RS*")):
        mask_fileRef = m
    return slices_folderRef, mask_fileRef

###Fonction en lien avec les fichiers

In [6]:

def createFile(nameFile, path):
    open(path + nameFile + '.xlsx','w')
    print("fichier créé :", nameFile)

def createAllFile(listNameFile, path):
    for nameFile in listNameFile:
        createFile(nameFile, path)

def createDataFrame(data):
    df = pd.DataFrame(data)
    return df

def addDataToFile(df, nameFile, path):
    df.to_excel(path + nameFile + '.xlsx')
    print("données ajoutées à :", nameFile)

def addAllDataToFile(listData, listNameFile, listSheetName, path):
    if(len(listData)==len(listNameFile)):
      for index in range (0, len(listData)):
          addDataToFile(listData[index]+',', listNameFile[index], listSheetName[index], path)
    else:
      print("les deux listes ne sont pas de même taille")

def createDicoVide(Listname, dico):
    for name in Listname:
        dico[name]={}
    return dico

def creaFileAddVal(nameFile, pathFileSave, dico):
    data = createDataFrame(dico)
    addDataToFile(data, nameFile, pathFileSave)

##Les fonctions de statistique

###Fonction pour les tests

In [7]:
def SSIM(imgRef, imgRec):
    return structural_similarity(imgRef, imgRec, data_range=(imgRec.max()-imgRec.min()))

def entropyImage(img, nameData, nameFile, pathFileSave, incrementation):
    ent = entropy(img, base=2)
    print(ent.shape)
    ent.shape=np.multiply(ent.shape[0], ent.shape[1])
    data = pd.DataFrame.from_dict({nameData: ent.tolist()})
    if incrementation !=0:
        x=pd.read_excel(pathFileSave + nameFile + '.xlsx', engine='openpyxl')
        dfNew=pd.concat([data,x])
        dfNew.to_excel(pathFileSave + nameFile + '.xlsx',index=False)
        del x, dfNew
    else:
      print("Fichier vide", os.path.getsize(pathFileSave + nameFile + '.xlsx'))
      data.to_excel(pathFileSave + nameFile + '.xlsx',index=False)
    del data, ent

def signalNoise(img):
    return stats.signaltonoise(img)

def SIMGradient(imgRef, imgRec):
    return structural_similarity(np.array(np.gradient(imgRef, axis=0)), np.array(np.gradient(imgRec, axis=0)), data_range=(imgRec.max()-imgRec.min()))

def mesureContrastAbsoluSignalBruit(img, mask, maskAround, zone, nameData, nameFile, pathFileSave):
    valLabel = np.where(mask==zone, img, 0)
    valLabel.shape=np.multiply(valLabel.shape[2],np.multiply(valLabel.shape[0], valLabel.shape[1]))
    valExterieur = np.where(maskAround==zone, img, 0)
    valExterieur.shape=np.multiply(valExterieur.shape[2],np.multiply(valExterieur.shape[0], valExterieur.shape[1]))

    sommeContrast = np.mean(valExterieur)- np.mean(valLabel)
    #allContrast = {nameData + "valExterne" : valExterieur.tolist(), nameData + "ValInterne" : valLabel.tolist()}
    SignalBruit = np.sum(valLabel)/np.sum(valExterieur) -1
    del valLabel, valExterieur
    return SignalBruit, sommeContrast

def pourcentageDiffPixel(maskRef, maskRec):
    return np.mean(((abs(np.subtract(maskRef, maskRec))/255)*100))

def volumeTumeur(mask):
    return np.sum(mask)*0.25

def auc_spec_sens_jaccard(maskRef, maskRec, zone=1.0):
    recZone = np.where(maskRec==zone , maskRec, 100000) #reconstruit vaut vrai
    refZone = np.where(maskRef==zone , maskRef, 100000) #reference vaut vrai
    invRecZone = np.where(maskRec!=zone , maskRec, 100000) #reconstruit vaut faux

    tpWhere=np.where(refZone==maskRec, 1,0)
    fnWhere=np.where(maskRec-maskRef==-1, 1,0) #reconstruit vaut faux, reference vrai
    fpWhere=np.where(maskRec-maskRef==1, 1,0) #reconstruit vaut vrai, reference faux
    tnWhere=np.where(invRecZone==maskRef, 1,0)

    print("truePositif", np.sum(tpWhere), "falseNegatif", np.sum(fnWhere), "falsePositif", np.sum(fpWhere), "trueNegatif", np.sum(tnWhere))
    sens= np.sum(tpWhere)/(np.sum(tpWhere)+np.sum(fnWhere))
    spec= np.sum(tnWhere)/(np.sum(tnWhere)+np.sum(fpWhere))

    jaccard = np.sum(tpWhere)/(np.sum(fnWhere)+np.sum(fpWhere)+np.sum(tpWhere))

    labelRef=np.where(maskRef==zone , 1, 0)
    labelRec=np.where(maskRec==zone , 1, 0)

    auc = roc_auc_score(labelRef.reshape(labelRef.shape[0]*labelRef.shape[1]*labelRef.shape[2]), labelRec.reshape(labelRec.shape[0]*labelRec.shape[1]*labelRec.shape[2]))
    del recZone, refZone, invRecZone, tpWhere, fnWhere, fpWhere, tnWhere
    return auc, spec, sens, jaccard



##La fonction final permettant de récupérer des dictionnaires pour chaque statistique

La durée jusqu'à la fin du calcul du SSIM pour un patient est de 24 seconde environ, se qui impact le plus sur le temps est l'appel de la fonction
readImageMasks

###Fonction avec des exels


In [ ]:
import cv2
def analyseInFile(patient_folder):
    #création des fichiers pour les différents test statistiques
    listNameSheetAll = ['SSIMImage', 'SSIMMask', 'SSIMGradientImage', 'PourcentageDiffPixels',
                        'Signal/Bruit', 'AUC','Specificite', 'Sencibilite', 'Jaccard', 'Contrast', 'Volume']
    dico={}
    createDicoVide(listNameSheetAll, dico)
    print(dico)
    listPatient=[]
    for patient in listdir(patient_folder):
        if patient[9:13]!= '_REC' :
            listPatient.append(patient[:-4])

    nameFile='Entropie_REF'
    createFile(nameFile, pathFileSave)
    nameFile='Entropie_REC'
    createFile(nameFile, pathFileSave)

    incrementation = 0
    #parcourt du dossier ou sont tout les patients
    for patient in tqdm(listPatient):
        #verification de la présence des fichiers
        if len(glob.glob(os.path.join(patient_folder, patient+ '_REF',  "RS*"))) != 0 and len(glob.glob(os.path.join(patient_folder, patient+ '_REF',  "MR*"))) != 0:
            slices_folderRef, mask_fileRef = miseDansMRFichier('_REF', patient_folder, patient)
            slices_folderRec, mask_fileRec = miseDansMRFichier('_REC', patient_folder, patient)

            #récupération des images et de la liste des masks
            print("pour la référence")
            imgRef, maskRef, maskRefName = readImageMasks(slices_folderRef, mask_fileRef)
            print("pour la reconstruite IA")
            imgRec, maskRec, maskRecName = readImageMasks(slices_folderRec, mask_fileRec)
            #print(imgRec.shape)
            imgRecS = np.empty([512, 512, 208])
            for i in range (0, 208):
                imgRecS[:,:,i] = cv2.resize(imgRec[:,:,i], (512, 512), interpolation=cv2.INTER_AREA)

            print("Noms des masques de REF pour le patient ", patient, " :", maskRefName, " et noms des masques de REC :", maskRecName)


            del slices_folderRec, slices_folderRef, mask_fileRec, mask_fileRef

            #passage des différents tests aux images et stokage dans la feuille exel respective
            for i in tqdm(range (0,1)):
              #print(imgRef.shape, np.resize(imgRec, (512, 512, 208)).shape)
              dico['SSIMImage'].update({patient: SSIM(imgRef, imgRecS)})
              np.seterr(invalid='ignore')
              entropyImage(imgRef, patient+'_REF', 'Entropie_REF', pathFileSave, incrementation)
              entropyImage(imgRec, patient+'_REC', 'Entropie_REC', pathFileSave, incrementation)
              dico['SSIMGradientImage'].update({patient:SIMGradient(imgRef, imgRecS)})

            #test sur les masques
            maskRecS = []
            masquePlacer = 0
            valeurRecS = 0
            for i in tqdm(range (0, len(maskRefName))):
                for j in range (0, len(maskRecName)):
                    if maskRecName[j] == maskRefName[i]:
                        masquePlacer = 1
                        maskRecS.append(np.empty([512, 512, 208]))
                        for valeur in tqdm(range (0, 208)):
                            maskRecS[valeurRecS][:,:,valeur] = cv2.resize(maskRec[j][:,:,valeur], (512, 512), interpolation=cv2.INTER_AREA)
                        #fig = plt.figure(figsize=(9,9))
                        #plt.imshow((maskRef[j][:,:,147]), cmap='gray')
                        #plt.show()
                        #fig = plt.figure(figsize=(9,9))
                        #plt.imshow((maskRec[j][:,:,147]), cmap='gray')
                        #plt.show()
                        signalBruitREF, contrastREF = mesureContrastAbsoluSignalBruit(imgRef, maskRef[i], dilatation(maskRef[i],6) - maskRef[i], 1.0,  patient+'_REF','Contrast_REF', pathFileSave)
                        dico['Contrast'].update({patient+maskRefName[i]+'_REF' : contrastREF})
                        dico['Signal/Bruit'].update({patient+maskRefName[i]+'_REF' : signalBruitREF})
                        del signalBruitREF, contrastREF

                        signalBruitREC, contrastREC = mesureContrastAbsoluSignalBruit(imgRec, maskRec[j], dilatation(maskRec[j],6) - maskRec[j], 1.0,  patient+'_REC','Contrast_REC', pathFileSave)
                        dico['Contrast'].update({patient+maskRecName[j]+'_REC' : contrastREC})
                        dico['Signal/Bruit'].update({patient+maskRecName[j]+'_REC' : signalBruitREC})
                        del signalBruitREC, contrastREC


                        dico['Volume'].update({patient+maskRefName[i]+'_REF' : volumeTumeur(maskRef[i])})
                        dico['Volume'].update({patient+maskRecName[j]+'_REC' : volumeTumeur(maskRec[j])})
                        dico['PourcentageDiffPixels'].update({patient: pourcentageDiffPixel(maskRef[i], maskRecS[valeurRecS])})

                        auc,Spec,Sens, jaccard = auc_spec_sens_jaccard(maskRef[i], maskRecS[valeurRecS], zone=1.0)
                        dico['AUC'].update({patient: auc})
                        dico['Specificite'].update({patient: Spec})
                        dico['Sencibilite'].update({patient: Sens})
                        dico['Jaccard'].update({patient: jaccard})
                        del auc,Spec,Sens, jaccard

                        dico['SSIMMask'].update({patient:SSIM(maskRef[i], maskRecS[valeurRecS])})
                if masquePlacer==0:
                    signalBruitREF, contrastREF = mesureContrastAbsoluSignalBruit(imgRef, maskRef[i], dilatation(maskRef[i],6) - maskRef[i], 1.0,  patient+'_REF','Contrast_REF', pathFileSave)
                    dico['Contrast'].update({patient+maskRefName[i]+'_REF' : contrastREF})
                    dico['Signal/Bruit'].update({patient+maskRefName[i]+'_REF' : signalBruitREF})
                    del signalBruitREF, contrastREF

                    dico['Volume'].update({patient+maskRefName[i]+'_REF' : volumeTumeur(maskRef[i])})
                masquePlacer=0
                valeurRecS+=1
        incrementation += 1
    return dico

nameFile='statistiquesRadiothérapeutes'
createFile(nameFile, pathFileSave)
dico=analyseInFile(patient_Folder)
data = createDataFrame(dico)
addDataToFile(data, nameFile, pathFileSave)

fichier créé : statistiquesRadiothérapeutes
{'SSIMImage': {}, 'SSIMMask': {}, 'SSIMGradientImage': {}, 'PourcentageDiffPixels': {}, 'Signal/Bruit': {}, 'AUC': {}, 'Specificite': {}, 'Sencibilite': {}, 'Jaccard': {}, 'Contrast': {}, 'Volume': {}}
fichier créé : Entropie_REF
fichier créé : Entropie_REC


  0%|          | 0/2 [00:00<?, ?it/s]

Le dossier MR existe déjà dans _REF
Le dossier MR existe déjà dans _REC
pour la référence


In [ ]:
#nameFile='statistiquesRadiothérapeutes'
#createFile(nameFile, pathFileSave)
#data = pd.DataFrame.from_dict(data)
#addDataToFile(data, nameFile, pathFileSave)
liste = [1,2,3,4]
for i in range(0,len(liste)):
    print(liste[i])

## Pour vider la mémoire ram:
Ne pas oublier de verifier que l'objet ne commence pas par '__' pour éviter de supprimer des variables qui ne sont pas les notres, juste en dessous on vérifie que les variables ont bien été supprimées

In [15]:
d = dir()
for obj in d:
    if not obj.startswith('__'):
        del globals()[obj]

##Les statistiques calculées ci-dessus:

###Pour les images:
*   SSIM
*   L'entropie
*   SSIM sur les gradients
*   Mesure du contrast absolu entre la tumeur et le tissu sain proche(environ même nombre de pixel) puis comparaison des deux
*   Signal sur bruit

###Pour les masques:
*   SSIM
*   ROC et AUC
*   Index de Jaccard entre les volumes
*   Distance pixel à pixel -> pourcentage de différence
*   Supperposition des masques Ref et Rec


###Les affichages:
(pour les graphes: y par x)
(on compare les graphes de Ref, Rec et ACC)
*   Histogramme avec fréquence des pixels par valeur des pixels dans le cerveau pour une frame + Print valeur moyenne, écart-type et nombre de pixels
*   Boîte à moustache pour 10 patients sur une frame dans le cerveau
*   Diagramme avec valeur des pixels par l'entropie
*   Distribution de pixel selon une droite + Print Skewness et Kurtosis
*   superposition des masques ref et rec en image pour evaluer les difference entre les deux
*   Graphe pour le signal sur bruit à determiner



Chaque données sont stocké dans des fichier sous forme d'un dictionnaire



##Tests de students


In [ ]:
import scipy
def deuxMoyenneTest(moy1,moy2):
    stat, pvalu = scipy.stats.ttest_rel(moy1, moy2)
    return pvalu
print("signal/bruit :", deuxMoyenneTest(
    [0.7324107885,-0.1047077179,0.6989234686,-0.2362701893,-0.3088008165,
     -0.7900961637, -0.3492358327],
     [0.1066627502,-0.4281967282,0.3133996725,-0.4741707444,-0.3987656236,
     -0.8560599238,-0.6173856258]))
print("contrast :", deuxMoyenneTest(
    [-4708143,216762,-20438788,467220,854424,740727,596528],
     [-239228,362445,-3233223,357245,550809,428226,447357]))
print("volume :", deuxMoyenneTest(
    [37474,6214,261356,5982,7985,7985,3336],
     [9198,2065,75061,1840,3971,3971,1007]))
print("qualitéImage :", deuxMoyenneTest(
    [2,2,2,3,3,3,2],
     [4,4,5,5,5,4,5]))
print("moyenne Q ref ", np.mean([4,4,5,5,5,4,5]))
print("moyenne Q rec ", np.mean([2,2,2,3,3,3,2]))

signal/bruit : 0.0073196140754231455
contrast : 0.26904064619713464
volume : 0.24297159893019524
qualitéImage : 0.0001754714755209395
moyenne Q ref  4.571428571428571
moyenne Q rec  2.4285714285714284


In [ ]:
img = [[0,0,0,0,0],[0,0,1,1,0],[1,0,1,1,0],[1,0,0,0,0],[0,0,0,0,0]]